In [1]:
import requests
import psycopg2
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Get data into a dataframe

api_url = "https://api.coingecko.com/api/v3/exchange_rates"
response = requests.get(api_url)

data = response.json()
rates = data['rates']

tempdf = pd.DataFrame.from_dict(rates)
df = tempdf.T

print(df)

                            name  unit        value       type
btc                      Bitcoin   BTC          1.0     crypto
eth                        Ether   ETH       13.562     crypto
ltc                     Litecoin   LTC      224.239     crypto
bch                 Bitcoin Cash   BCH      156.738     crypto
bnb                 Binance Coin   BNB       59.832     crypto
...                          ...   ...          ...        ...
xdr   IMF Special Drawing Rights   XDR    12921.382       fiat
xag          Silver - Troy Ounce   XAG      729.452  commodity
xau            Gold - Troy Ounce   XAU        9.529  commodity
bits                        Bits  μBTC    1000000.0     crypto
sats                     Satoshi  sats  100000000.0     crypto

[61 rows x 4 columns]


In [3]:
# Connect to RDS Postgres database
conn = psycopg2.connect(
    database="crypto",
    user="jkamino",
    password="Tacoman123",
    host="data-5330-1.ckpcm5tpab0e.us-west-1.rds.amazonaws.com",
    port="5432")

cursor = conn.cursor() 

In [4]:
# Create Table if it doesn't exist
query = """
CREATE TABLE IF NOT EXISTS exchange_rates (
  exchange_id serial PRIMARY KEY,
  name VARCHAR(100) NOT NULL,
  unit VARCHAR(5) NOT NULL,
  value FLOAT, 
  type VARCHAR(20), 
  date TIMESTAMP  
);
"""

cursor.execute(query)
conn.commit()

In [5]:
# Loop through all items in df, ingest
for x in df.index:
    cursor.execute(f"""
                    INSERT INTO exchange_rates ( name, unit, value, type, date)
                    VALUES ('{df.loc[x]["name"]}', '{df.loc[x]["unit"]}', '{df.loc[x]["value"]}', '{df.loc[x]["type"]}', current_timestamp)
                    """)

    conn.commit()
    
    name = df.loc[x]["name"]
    
    print(f"{name} ingested")
    

Bitcoin ingested
Ether ingested
Litecoin ingested
Bitcoin Cash ingested
Binance Coin ingested
EOS ingested
XRP ingested
Lumens ingested
Chainlink ingested
Polkadot ingested
Yearn.finance ingested
US Dollar ingested
United Arab Emirates Dirham ingested
Argentine Peso ingested
Australian Dollar ingested
Bangladeshi Taka ingested
Bahraini Dinar ingested
Bermudian Dollar ingested
Brazil Real ingested
Canadian Dollar ingested
Swiss Franc ingested
Chilean Peso ingested
Chinese Yuan ingested
Czech Koruna ingested
Danish Krone ingested
Euro ingested
British Pound Sterling ingested
Hong Kong Dollar ingested
Hungarian Forint ingested
Indonesian Rupiah ingested
Israeli New Shekel ingested
Indian Rupee ingested
Japanese Yen ingested
South Korean Won ingested
Kuwaiti Dinar ingested
Sri Lankan Rupee ingested
Burmese Kyat ingested
Mexican Peso ingested
Malaysian Ringgit ingested
Nigerian Naira ingested
Norwegian Krone ingested
New Zealand Dollar ingested
Philippine Peso ingested
Pakistani Rupee inges

In [9]:
# Lets see if this worked
query = (f"""
SELECT *
FROM exchange_rates
""")

df_sql_table = pd.read_sql(query, conn,index_col='exchange_id')

print(df)

df_sql_table.to_csv("Database-content.csv")

                            name  unit        value       type
btc                      Bitcoin   BTC          1.0     crypto
eth                        Ether   ETH       13.562     crypto
ltc                     Litecoin   LTC      224.239     crypto
bch                 Bitcoin Cash   BCH      156.738     crypto
bnb                 Binance Coin   BNB       59.832     crypto
...                          ...   ...          ...        ...
xdr   IMF Special Drawing Rights   XDR    12921.382       fiat
xag          Silver - Troy Ounce   XAG      729.452  commodity
xau            Gold - Troy Ounce   XAU        9.529  commodity
bits                        Bits  μBTC    1000000.0     crypto
sats                     Satoshi  sats  100000000.0     crypto

[61 rows x 4 columns]


## Remote execution

I uploaded my code to a EC2 instance and set up a cron task to run my code every 6 hours. 

I downloaded my code to the EC2 using my Github link which can be found here: https://github.com/Jkamin0/IS3600

Once downloaded I set up cron to run using the crontab -e command and specifying my file location and to run ever 6 hours. The logic to run ever 6 hours in cron was set using: 0 */6 * * *.

## Project Takeaways

Overall, this was a super fun project to work on and learn more about the services offered by AWS. I personally think that the RDS database system is super helpful and I'm sure I'll be working with it in the future in my career. I think a lot of features AWS offeres you could feasibly run and maintain on your own. However, managing a database I feel would be super stressful and is a great choice for something to hand off to a big company to deal with. 

I'd also never worked with cron before this project, which is a great skill to have experience with. Getting it up and running was a big of a speed bump but there are lots of great guides online. I used this super great site to set up how often I wanted the cron job to run: https://crontab.guru/